In [1]:
import logging
import os
import re
from typing import Any, Container, Dict, Iterable, List, Optional, TextIO, Union, cast
import unicodedata

import pdfminer
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines, PDFXRefFallback
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdftypes import PDFObjectNotFound, PDFValueError
from pdfminer.pdftypes import PDFStream, PDFObjRef, resolve1, stream_value
from pdfminer.psparser import PSKeyword, PSLiteral, LIT
from pdfminer.utils import isnumber

In [2]:
logging.basicConfig()
logger = logging.getLogger(__name__)

ESC_PAT = re.compile(r'[\000-\037&<>()"\042\047\134\177-\377]')
def escape(s: Union[str, bytes]) -> str:
    if isinstance(s, bytes):
        us = str(s, "latin-1")
    else:
        us = s
    return ESC_PAT.sub(lambda m: "&#%d;" % ord(m.group(0)), us)

def bintostr(b:bytes):
    if b is None:
        return ""
    return unicodedata.normalize('NFKD', b.decode("latin-1")).encode('ASCII', 'ignore').decode("ASCII")

In [3]:
def processPdfAndSaveTxt(pdfname: str) -> None:
    retrievedData=processPdf(pdfname)
    ## Save map to file
    outfilename=pdfname + ".txt"
    outfp=open(outfilename , "w")
    for k,v in sorted(retrievedData.items()):
        outfp.write("%s=%s\n" % (k, v.strip().replace("\r\n", ".")))
    outfp.close()

In [4]:
def processPdf(
    fname: str
) -> dict:
    retrievedData = {}
    fp = open(fname, "rb")
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    searchInAllObjects(retrievedData, doc)
    fp.close()
    return retrievedData

In [5]:
def searchInAllObjects(
    retrievedData: dict,
    doc: PDFDocument
) -> None:
    visited = set()
    for xref in doc.xrefs:
        for objid in xref.get_objids():
            if objid in visited:
                continue
            visited.add(objid)
            try:
                obj = doc.getobj(objid)
                if obj is None:
                    continue
                searchInObject(retrievedData, obj, objid=objid)
            except PDFObjectNotFound as e:
                print("not found: %r" % e)
    return

In [6]:
def searchInObject(
    retrievedData: dict,
    obj: object, 
    objid: str = None
) -> None:
    understood=False
    if isinstance(obj, dict) and 'Type' in obj.keys():
        type=obj["Type"]
        if isinstance(type, PSLiteral):
            type=type.name
        #
        if type=="Annot":
            if 'T' in obj.keys() and 'V' in obj.keys():
                valueT=bintostr(obj["T"])
                valueV=bintostr(obj["V"])
                #
                if 'Opt' in obj.keys(): ##Combo
                    value=""
                    if isinstance(obj["Opt"], list):
                        for nestedList in obj["Opt"]:
                            if isinstance(nestedList, list) and len(nestedList)==2:
                                try:
                                    nestedListKey=bintostr(nestedList[0])
                                except:
                                    out.write('!! Error decoding nestedlist key %s \n' % nestedList[0])
                                try:
                                    nestedListValue=bintostr(nestedList[1])
                                except:
                                    out.write('!! Error decoding nestedlist value %s \n' % nestedList[1])
                                if nestedListKey==valueV:
                                    value=nestedListValue
                    understood=True
                    retrievedData[valueT]=value
                else: ##Campo de texto
                    understood=True
                    retrievedData[valueT]=valueV
        elif type in ["Pages", "Page", "Font", "FontDescriptor", "Encoding", "Outlines", "Catalog"]:
            understood=True
        else: ## type desconocido
            understood=True
            print("<unknown type=\"%s\"/>\n" % type)
    
if __name__ == "__main__":
    for file in os.listdir("."):
        if file.endswith(".pdf"):
            fileName=os.path.join(".", file)
            print("Processing %s" % fileName)
            processPdfAndSaveTxt(fileName)
    print("Done")


Processing .\Test - Copy.pdf
Processing .\Test.pdf
Done
